# CDMX Neighbors Clustering with Foursquare API

In this blog we will build a k-means clustering of neighbors in CDMX using data of venues registered on Foursquare API.

CDMX, short for Ciudad de México, or Mexico City, is the capital and largest city of Mexico. With a population of over 21 million people in the metropolitan area, it is one of the most populous cities in the world. As with many densely populated urban areas, CDMX faces various challenges related to urban density and its impact on the cost of rents.

Some factors like Housing Demand vs Supply, Gentrification, Limited Space and Land, Transportation and Accessibility, Income Disparities, informal Settlements and rent control or regulations make a huge challenge to find an apartment in the City.

Addressing the challenges of urban density and the cost of rents in CDMX requires a comprehensive approach that involves increasing the supply of affordable housing, investing in public transportation to improve accessibility, promoting sustainable urban planning, and implementing policies that support economic development and income equality.

This study is for all new people that arrive in CDMX for a job. It could be helpful to have an idea of where you want to live.
I was new in the City some years ago, and to find rent is a headache when you have not been in CDMX before.

## 1. Get all ready 

Import Libraries

In [1]:
import numpy as np # librería para manejar datos vectorizados

import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from collections import Counter

import json # librería para manejar archivos JSON 

#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Libraries imported.


## 2. Download and explore the data set

To make it easy I will share with you the data used for this example in the follow link:
Original data was gathered from Correos México


In [2]:
neighborhoods = pd.read_csv('./data_cdmx_planos.csv', encoding='ISO-8859-1')
neighborhoods.head()

,City,Borough,Neighborhood,CP,Latitude,Longitude
0,cdmx,miguel hidalgo,Anzures,11590,19.427472,-99.177193
1,cdmx,miguel hidalgo,Argentina,11230,19.455959,-99.203590
2,cdmx,miguel hidalgo,Bosque de Chapultepec,11100,19.421800,-99.188193
3,cdmx,miguel hidalgo,Bosque de las Lomas,11619,19.407364,-99.240680
4,cdmx,miguel hidalgo,Daniel Garza,11830,19.410295,-99.192390


And let's make sure that the data set has the 3 boroughs and the 57 neighboorhods.

In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 13 boroughs and 111 neighborhoods.


## 3. Data Visualization

We will use the geopy library to obtain the latitude and longitude of Mexico City

In order to define an instance of the geocoder we will need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [4]:
address = 'Mexico City, CDMX'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Mexico City, CDMX are 19.2667063, -99.1269596.


We will generate a map of CDMX with the neighborhoods on it.

In [6]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_cdmx = folium.Map(location=[latitude, longitude], zoom_start=7)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cdmx)  
    
map_cdmx

## 4. Using Foursquare API

Now we are going to use the Foursquare API to collect information from each neighborhood

Foursquare API (Application Programming Interface) is a set of tools and protocols that allows developers to access and interact with data from Foursquare's location-based platform. Foursquare is a popular location intelligence company that provides location data and services to various applications and developers.

The API enables developers to build location-based applications and services by accessing a wealth of location-related information, including places, venues, user check-ins, tips, reviews, and more. This data can be used to create location-aware applications, social networking features, personalized recommendations, and location-based marketing campaigns.

In [7]:
headers = {"Accept":"application/json",
          "Authorization":" your api-key here"}

In [8]:
# función para extraer la categoria del sitio
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
def getNearbyVenues(names, latitudes, longitudes, headers, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = "https://api.foursquare.com/v3/places/search"
        search_query=''
        params = {
        "ll": str(lat)+","+str(lng),
        "radius": 500,
        "limit": 50, # 50 son el numero máximo para la version3 de Foursquare aPI
        "sort":"DISTANCE"}
            
        # solicitud GET
        response = requests.request("GET", url, params=params, headers=headers)
        results = response.json()
        results = results['results']

        # regresa solo información relevante de cada sitio cercano
        for venue in results:
                try:
                    venues_list.append([name.strip(), lat,lng, 
                                        venue['name'], 
                                        venue['geocodes']['main']['latitude'],
                                        venue['geocodes']['main']['longitude'],
                                        venue['categories'][0]['name']])
                except Exception as e:
                    venues_list.append([name.strip(), lat,lng, 
                                        venue['name'], 
                                        venue['geocodes']['main']['latitude'],
                                        venue['geocodes']['main']['longitude'],
                                        'Not Especified'])
    print(venues_list)
    #nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues = pd.DataFrame(venues_list)
    print(nearby_venues)
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We consulted the information of the 50 registered places of each neighboorhod

In [10]:
# Run the getNearbyVenues for each neighboorhod on the dataframe
cdmx_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],headers= headers)

Anzures
Argentina
Bosque de Chapultepec
Bosque de las Lomas
Daniel Garza
Lago Sur
Tacuba
Lomas Altas
Lomas de Chapultepec
Lomas Hermosa
Mexico Nuevo
Pensil
Polanco
Popotla
Reforma Social
San Miguel Chapultepec
Santo Tomas
Tacubaya
Tlaxpana
Atlampa
Santa Maria insurgentes
Peralvillo
Ex-Hipodromo de Peralvillo
Valle Gomez
Maza
Nonoalco Tlatelolco
Morelos
Guerrero
Buenavista
Santa Maria la Ribera
San Rafael
Tabacalera
Cuauhtemoc
Juarez
Roma Norte
Doctores
Obrera
Condesa
Hipodromo Condesa
Hipodromo
Roma Sur
Algarin
Transito
Ampliacion Asturias
Narvarte Poniente
Narvarte Oriente
Del Valle norte
Napoles
Mixcoac
San Pedro de los pinos
del valle centro
del valle sur
Centro (area 1)
Centro (area 2)
Centro (area 3)
Centro (area 4)
Agrícola Pantitlán
Agrícola Oriental
Granjas México
San Pedro
Colina del Sur
El Cuernito
Florida
Jardines del Pedregal
Lomas de La Era
Lomas de Plateros
Lomas de San Ángel Inn
San Ángel
San Bartolo Ameyalco
San Pedro de los Pinos
Santa Rosa Xochiac
Tarango
El Rosario
O

In [11]:
# print the final dataframe

print(cdmx_venues.shape)
cdmx_venues.head()

(5389, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Anzures,19.427472,-99.177193,Carrito de Dulces,19.427520,-99.177600,Food Truck
1,Anzures,19.427472,-99.177193,Selene,19.427104,-99.177391,Mexican Restaurant
2,Anzures,19.427472,-99.177193,Bodegas Alianza Anzurez,19.427144,-99.177766,Winery
3,Anzures,19.427472,-99.177193,Taqueria Selene,19.427132,-99.177324,Taco Restaurant
4,Anzures,19.427472,-99.177193,Italian-CITO,19.427132,-99.177324,Dining and Drinking


In [12]:
# clean the venues with category = "Not Especified"
cdmx_venues = cdmx_venues[cdmx_venues['Venue Category'] != 'Not Especified']

Let's check the number of places found for the first 5 neighboorhods

In [14]:
cdmx_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agrícola Oriental,47,47,47,47,47,47
Agrícola Pantitlán,47,47,47,47,47,47
Algarin,49,49,49,49,49,49
Ampliacion Asturias,49,49,49,49,49,49
Anzures,50,50,50,50,50,50


Let's find the unique categories of venues from the dataframe

In [15]:
print('There are {} uniques categories.'.format(len(cdmx_venues['Venue Category'].unique())))

There are 393 uniques categories.


## 5. Data Analysis


Let's analyze each neighborhood based on the frequency of appearance of each category of places found by Foursquare

In [16]:
# codificación
cdmx_onehot = pd.get_dummies(cdmx_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
cdmx_onehot['Neighborhood'] = cdmx_venues['Neighborhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [cdmx_onehot.columns[-1]] + list(cdmx_onehot.columns[:-1])
cdmx_onehot = cdmx_onehot[fixed_columns]

cdmx_onehot.head()

,Neighborhood,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Afghan Restaurant,Agriculture and Forestry Service,Alternative Medicine Clinic,American Restaurant,Amusement Park,Animal Shelter,Antique Store,Aquarium,Arcade,Architecture Firm,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Assisted Living Service,Automotive Repair Shop,Automotive Service,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bicycle Store,Big Box Store,Bike Trail,Bistro,Board Store,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Store,Bridge,Buffet,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafe, Coffee, and Tea House",Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Dealership,Car Parts and Accessories,Caribbean Restaurant,Carpenter,Carpet and Flooring Contractor,Casino,Cemetery,Cheese Store,Chemicals and Gasses Manufacturer,Child Care Service,Children's Clothing Store,Chinese Restaurant,Chocolate Store,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College and University,Comedy Club,Comfort Food Restaurant,Commercial Real Estate Developer,Community Center,Community and Government,Computer Repair Service,Concert Hall,Construction,Construction Supplies Store,Convenience Store,Cosmetics Store,Country Dance Club,Courthouse,Creperie,Cultural Center,Cupcake Shop,Daycare,Deli,Dentist,Department Store,Dermatologist,Dessert Shop,Diner,Dining and Drinking,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Park,Donut Shop,Drugstore,Education,Electronics Store,Elementary School,Embassy or Consulate,Emergency Room,Engineer,Equipment Rental Service,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Fair,Falafel Restaurant,Farm,Farmers Market,Fashion Accessories Store,Fast Food Restaurant,Financial Planner,Financial Service,Fireworks Store,Fish and Chips Shop,Flea Market,Floating Market,Flower Store,Fondue Restaurant,Food Court,Food Stand,Food Truck,Food and Beverage Retail,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit and Vegetable Store,Funeral Home,Furniture and Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Contractor,General Surgeon,Geological Service,German Restaurant,Gift Store,Golf Course,Gourmet Store,Government Building,Greek Restaurant,Grocery Store,Gun Store,Gym and Studio,Gymnastics Center,Harbor or Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Health and Beauty Service,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",Herbs and Spices Store,High School,Hiking Trail,Historic and Protected Site,History Museum,Hobby Store,Home Appliance Store,Home Improvement Service,Hookah Bar,Hospital,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Housewares Store,Housing Development,Human Resources Agency,Ice Cream Parlor,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Industrial Equipment Supplier,Insurance Agency,Interior Designer,Internet Cafe,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Laboratory,Lake,Landmarks and Outdoors,Language School,Latin American Restaurant,Law Office,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Locksmith,Lounge,Luggage Store,Machine Shop,Manufacturer,Martial Arts Dojo,Mattress Store,Meat and Seafood Store,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Mental Health Clinic,Metals Supplier,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Store,Mobile Company,Mobile Phone Store,Monastery,Monument,Moroccan Restaurant,Mosque,Motorcycle Dealership,Motorcycle Repair Shop,Mountain,Movie Theater,Museum,Music School,Music Store,Music Venue

One-hot encoding will help us to get a numeric value by that our clustering algorithm will learn

In [17]:
cdmx_onehot.shape

(5023, 394)

Let's group the rows by neighborhoods taking the average of the frequency of occurrence of each category

In [20]:
cdmx_grouped = cdmx_onehot.groupby('Neighborhood').mean().reset_index()
cdmx_grouped.head()

,Neighborhood,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Afghan Restaurant,Agriculture and Forestry Service,Alternative Medicine Clinic,American Restaurant,Amusement Park,Animal Shelter,Antique Store,Aquarium,Arcade,Architecture Firm,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Assisted Living Service,Automotive Repair Shop,Automotive Service,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bicycle Store,Big Box Store,Bike Trail,Bistro,Board Store,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Store,Bridge,Buffet,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafe, Coffee, and Tea House",Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Dealership,Car Parts and Accessories,Caribbean Restaurant,Carpenter,Carpet and Flooring Contractor,Casino,Cemetery,Cheese Store,Chemicals and Gasses Manufacturer,Child Care Service,Children's Clothing Store,Chinese Restaurant,Chocolate Store,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College and University,Comedy Club,Comfort Food Restaurant,Commercial Real Estate Developer,Community Center,Community and Government,Computer Repair Service,Concert Hall,Construction,Construction Supplies Store,Convenience Store,Cosmetics Store,Country Dance Club,Courthouse,Creperie,Cultural Center,Cupcake Shop,Daycare,Deli,Dentist,Department Store,Dermatologist,Dessert Shop,Diner,Dining and Drinking,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Park,Donut Shop,Drugstore,Education,Electronics Store,Elementary School,Embassy or Consulate,Emergency Room,Engineer,Equipment Rental Service,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Fair,Falafel Restaurant,Farm,Farmers Market,Fashion Accessories Store,Fast Food Restaurant,Financial Planner,Financial Service,Fireworks Store,Fish and Chips Shop,Flea Market,Floating Market,Flower Store,Fondue Restaurant,Food Court,Food Stand,Food Truck,Food and Beverage Retail,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit and Vegetable Store,Funeral Home,Furniture and Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Contractor,General Surgeon,Geological Service,German Restaurant,Gift Store,Golf Course,Gourmet Store,Government Building,Greek Restaurant,Grocery Store,Gun Store,Gym and Studio,Gymnastics Center,Harbor or Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Health and Beauty Service,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",Herbs and Spices Store,High School,Hiking Trail,Historic and Protected Site,History Museum,Hobby Store,Home Appliance Store,Home Improvement Service,Hookah Bar,Hospital,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Housewares Store,Housing Development,Human Resources Agency,Ice Cream Parlor,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Industrial Equipment Supplier,Insurance Agency,Interior Designer,Internet Cafe,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Laboratory,Lake,Landmarks and Outdoors,Language School,Latin American Restaurant,Law Office,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Locksmith,Lounge,Luggage Store,Machine Shop,Manufacturer,Martial Arts Dojo,Mattress Store,Meat and Seafood Store,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Mental Health Clinic,Metals Supplier,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Store,Mobile Company,Mobile Phone Store,Monastery,Monument,Moroccan Restaurant,Mosque,Motorcycle Dealership,Motorcycle Repair Shop,Mountain,Movie Theater,Museum,Music School,Music Store,Music Venue

for a better visualization, let's print the top 5 of venues for each neighborhood

In [21]:
num_top_venues = 5

for hood in cdmx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cdmx_grouped[cdmx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agrícola Oriental----
             venue  freq
0  Taco Restaurant  0.09
1         Hospital  0.04
2           Retail  0.04
3   Mobile Company  0.04
4    Grocery Store  0.04


----Agrícola Pantitlán----
                      venue  freq
0             Grocery Store  0.11
1  Furniture and Home Store  0.09
2           Taco Restaurant  0.09
3       Government Building  0.06
4     Sports and Recreation  0.06


----Algarin----
                venue  freq
0      Clothing Store  0.20
1         Print Store  0.14
2    Department Store  0.08
3     Taco Restaurant  0.08
4  Mexican Restaurant  0.06


----Ampliacion Asturias----
                venue  freq
0       Grocery Store  0.08
1  Mexican Restaurant  0.08
2             Dentist  0.06
3         Coffee Shop  0.06
4      Breakfast Spot  0.04


----Anzures----
                 venue  freq
0   Mexican Restaurant  0.12
1                 Café  0.08
2  Dining and Drinking  0.08
3        Grocery Store  0.06
4      Taco Restaurant  0.04


----Argentina

Generate a new dataframe with this info

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cdmx_grouped['Neighborhood']

for ind in np.arange(cdmx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cdmx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agrícola Oriental,Taco Restaurant,Hospital,Retail,Mobile Company,Grocery Store,Candy Store,Convenience Store,Medical Lab,Fast Food Restaurant,Coffee Shop
1,Agrícola Pantitlán,Grocery Store,Furniture and Home Store,Taco Restaurant,Government Building,Sports and Recreation,Doctor's Office,Performing Arts Venue,Food Truck,Construction Supplies Store,Farm
2,Algarin,Clothing Store,Print Store,Department Store,Taco Restaurant,Mexican Restaurant,Miscellaneous Store,Shopping Mall,Office Supply Store,Arts and Crafts Store,Food Truck
3,Ampliacion Asturias,Grocery Store,Mexican Restaurant,Dentist,Coffee Shop,Breakfast Spot,Café,Bar,Food Court,Fried Chicken Joint,Miscellaneous Store
4,Anzures,Mexican Restaurant,Café,Dining and Drinking,Grocery Store,Taco Restaurant,Burger Joint,Spanish Restaurant,Sushi Restaurant,Convenience Store,Temple


Codify the info to the final feature set

In [32]:
cdmx_grouped_clustering = cdmx_grouped.drop('Neighborhood', 1)
cdmx_grouped_clustering.head()

C:\Users\Asus.DESKTOP-TJAT9N3\AppData\Local\Temp\ipykernel_21120\1976807158.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  cdmx_grouped_clustering = cdmx_grouped.drop('Neighborhood', 1)


,Accounting and Bookkeeping Service,Adult Store,Advertising Agency,Afghan Restaurant,Agriculture and Forestry Service,Alternative Medicine Clinic,American Restaurant,Amusement Park,Animal Shelter,Antique Store,Aquarium,Arcade,Architecture Firm,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Assisted Living Service,Automotive Repair Shop,Automotive Service,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bicycle Store,Big Box Store,Bike Trail,Bistro,Board Store,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Store,Bridge,Buffet,Burger Joint,Burrito Restaurant,Business and Professional Services,Business and Strategy Consulting Office,Butcher,"Cafe, Coffee, and Tea House",Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Car Dealership,Car Parts and Accessories,Caribbean Restaurant,Carpenter,Carpet and Flooring Contractor,Casino,Cemetery,Cheese Store,Chemicals and Gasses Manufacturer,Child Care Service,Children's Clothing Store,Chinese Restaurant,Chocolate Store,Church,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College and University,Comedy Club,Comfort Food Restaurant,Commercial Real Estate Developer,Community Center,Community and Government,Computer Repair Service,Concert Hall,Construction,Construction Supplies Store,Convenience Store,Cosmetics Store,Country Dance Club,Courthouse,Creperie,Cultural Center,Cupcake Shop,Daycare,Deli,Dentist,Department Store,Dermatologist,Dessert Shop,Diner,Dining and Drinking,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Park,Donut Shop,Drugstore,Education,Electronics Store,Elementary School,Embassy or Consulate,Emergency Room,Engineer,Equipment Rental Service,Escape Room,Ethiopian Restaurant,Event Service,Exhibit,Eyecare Store,Fair,Falafel Restaurant,Farm,Farmers Market,Fashion Accessories Store,Fast Food Restaurant,Financial Planner,Financial Service,Fireworks Store,Fish and Chips Shop,Flea Market,Floating Market,Flower Store,Fondue Restaurant,Food Court,Food Stand,Food Truck,Food and Beverage Retail,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit and Vegetable Store,Funeral Home,Furniture and Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Contractor,General Surgeon,Geological Service,German Restaurant,Gift Store,Golf Course,Gourmet Store,Government Building,Greek Restaurant,Grocery Store,Gun Store,Gym and Studio,Gymnastics Center,Harbor or Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Health and Beauty Service,Health and Medicine,Healthcare Clinic,"Heating, Ventilating and Air Conditioning Contractor",Herbs and Spices Store,High School,Hiking Trail,Historic and Protected Site,History Museum,Hobby Store,Home Appliance Store,Home Improvement Service,Hookah Bar,Hospital,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Housewares Store,Housing Development,Human Resources Agency,Ice Cream Parlor,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Industrial Equipment Supplier,Insurance Agency,Interior Designer,Internet Cafe,Italian Restaurant,Japanese Restaurant,Jazz and Blues Venue,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Laboratory,Lake,Landmarks and Outdoors,Language School,Latin American Restaurant,Law Office,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Locksmith,Lounge,Luggage Store,Machine Shop,Manufacturer,Martial Arts Dojo,Mattress Store,Meat and Seafood Store,Medical Center,Medical Lab,Medical Supply Store,Mediterranean Restaurant,Mental Health Clinic,Metals Supplier,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Miscellaneous Store,Mobile Company,Mobile Phone Store,Monastery,Monument,Moroccan Restaurant,Mosque,Motorcycle Dealership,Motorcycle Repair Shop,Mountain,Movie Theater,Museum,Music School,Music Store,Music Venue,National Par

print the final list of category venues founded by the top10

In [33]:
cdmx_grouped_clustering.dtypes

Accounting and Bookkeeping Service                        float64
Adult Store                                               float64
Advertising Agency                                        float64
Afghan Restaurant                                         float64
Agriculture and Forestry Service                          float64
Alternative Medicine Clinic                               float64
American Restaurant                                       float64
Amusement Park                                            float64
Animal Shelter                                            float64
Antique Store                                             float64
Aquarium                                                  float64
Arcade                                                    float64
Architecture Firm                                         float64
Arepa Restaurant                                          float64
Argentinian Restaurant                                    float64
Art Galler

## 6. Clustering CDMX Neighborhoods

Defining the kmeans configuration and train the algorithm

In [29]:
# establecer el número de agrupaciones
kclusters = 6

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=10).fit(cdmx_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_

C:\Users\Asus.DESKTOP-TJAT9N3\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


array([4, 0, 5, 4, 1, 2, 2, 5, 0, 5, 0, 5, 5, 5, 4, 4, 5, 5, 0, 1, 2, 4,
       4, 4, 4, 5, 4, 4, 4, 0, 4, 0, 1, 5, 5, 2, 0, 0, 1, 4, 2, 0, 2, 0,
       2, 2, 2, 4, 0, 4, 3, 2, 4, 4, 5, 2, 1, 0, 1, 4, 0, 4, 1, 3, 4, 1,
       4, 0, 2, 0, 2, 5, 2, 4, 4, 1, 0, 0, 4, 5, 4, 4, 2, 4, 5, 1, 4, 1,
       2, 4, 5, 4, 1, 4, 2, 2, 2, 1, 2, 0, 4, 4, 1, 1, 4, 2, 4, 0, 5, 4])

Clustering the original data

In [34]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_)

cdmx_merged = neighborhoods

# juntar manhattan_grouped con manhattan_data 
cdmx_merged = cdmx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cdmx_merged.head(6) # revisar las ultimas columnas

,City,Borough,Neighborhood,CP,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,cdmx,miguel hidalgo,Anzures,11590,19.427472,-99.177193,1.0,Mexican Restaurant,Café,Dining and Drinking,Grocery Store,Taco Restaurant,Burger Joint,Spanish Restaurant,Sushi Restaurant,Convenience Store,Temple
1,cdmx,miguel hidalgo,Argentina,11230,19.455959,-99.203590,2.0,Taco Restaurant,Mexican Restaurant,Church,Burger Joint,Office Building,Housing Development,Other Great Outdoors,Beer Garden,Burrito Restaurant,Stationery Store
2,cdmx,miguel hidalgo,Bosque de Chapultepec,11100,19.421800,-99.188193,5.0,Zoo,Park,Fast Food Restaurant,Amusement Park,Burger Joint,Arts and Entertainment,Playground,Landmarks and Outdoors,Government Building,Garden
3,cdmx,miguel hidalgo,Bosque de las Lomas,11619,19.407364,-99.240680,0.0,Office Building,Mexican Restaurant,Doctor's Office,Pizzeria,Mobile Company,Taco Restaurant,Food Court,Food Truck,Coffee Shop,Bakery
4,cdmx,miguel hidalgo,Daniel Garza,11830,19.410295,-99.192390,2.0,Taco Restaurant,Food Truck,Dining and Drinking,Drugstore,Housing Development,Mexican Restaurant,Seafood Restaurant,Snack Place,Miscellaneous Store,Synagogue
5,cdmx,miguel hidalgo,Lago Sur,11460,19.445285,-99.188345,2.0,Taco Restaurant,Mexican Restaurant,Candy Store,Bakery,Burger Joint,Dessert Shop,French Restaurant,Grocery Store,Butcher,Bar


## 7. Visualizate the results of clustering

In [35]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array =['blue','green','red','yellow','black','orange']

# añadir marcadores al mapa
markers_colors = []

for lat, lon, poi, cluster in zip(cdmx_merged['Latitude'], cdmx_merged['Longitude'], cdmx_merged['Neighborhood'], cdmx_merged['Cluster Labels']):
    try:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=colors_array[int(cluster)],
            fill=True,
            fill_color=colors_array[int(cluster)],
            fill_opacity=0.7).add_to(map_clusters)
    except Exception as e:
        print(e)
       
map_clusters

cannot convert float NaN to integer
